<a href="https://colab.research.google.com/github/palindrom615/notebooks/blob/main/grad/knockoffnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knockoff Nets

2014-11437 장회문

## Introduction

inference attack(?)에 관한 선행연구가 있었고, 본 연구에서는 선행연구와 달리 blackbox인 victim model에 대한 최소한의 가정을 바탕으로 기능을 훔치기만 하는 것에 집중. 기본적으로 (activethief랑 동일하게) thief dataset을 구축하고 victim model에 쿼링해서 받아온 값을 레이블로 삼아 knockoff model을 트레이닝시킴.

## Related Work

* cv쪽 privacy 관련 연구
* model의 정확한 재현을 목적으로 하는 model stealing
* KD
* Acive Learning - 이터레이션 돌 때 배치를 무작위로 고르는게 아니라 가장 효과적인 데이터들을 샘플링해 트레이닝시키기

## Problem statement

* 목적은 모델 $F_V$와 같은 기능을 하는 $F_A$를 트레이닝시키기
* victim은 secret data $P_V(X)$로 본인의 모델을 훈련시키고 이걸로 누군가 inference를 할 때마다 비용이 듦
* K dim posterior probability가 아웃풋인 victim model에 대해서 adversary는 1. 모델의 구조와 hyperparameters, 2. victim의 데이터셋, 3. K개의 클래스가 뭘 뜻하는지를 모른다.
* adversary는 모델 $F_A$를 설계하고, victim model에 질의할 thief dataset을 만들고, victim model의 아웃풋을 데이터의 레이블로 삼고, 그걸로 모델을 트레이닝시킨다.
* 추가로 최적화 대상은 1. B번 이하의 쿼리로 최대의 퍼포먼스 2. blackbox에 질의할 최적의 이미지 set 찾기
* victim은 rounding off 등으로 정보량을 줄일 수 있다.

KD와의 차이점: KD는 스승 모델에 쓰인 데이터셋과 참값을 쓸 수 있지만 knockoffnet은 아니다.



## Generating Knockoffs

### TransferSet 구축

전체 dataset(=$P_A(X)$) 선택 (큰 public imageset e.g. ILSVRC)
그 중 victim model에 질의할 것을 선택. 선택하는 방법은 여기서는 두가지 정도를 제시.
  * Random strategy
  * Adaptive strategy - 트레이닝 뒤 나온 근사값을 reward signal에 넣고 그걸 training set을 선택하는데 반영(?)
    * 주어진 dataset의 원래 레이블 값인 $z_i$를 사용하는 것이 학습에 도움이 됨. 클러스터링 알고리즘 등 다른 기법도 사용 가능.
    * 질의에 사용될 data는 각각의 $z$에서 $softmax(H_t(z)) = \pi_t(z)$의 확률로 섞어서 샘플링. 
    * 각 node potential $H_t(z)$를 gradient bandit algorithm으로 업데이트. 
    * reward로 certainty, diversity, loss 이렇게 세 개를 사용

### 훈련시키기

KD나 model compression 등의 연구에서 나오듯이 VGG나 Resnet 정도의 적당한 네트워크만 구축하면 되고, 꼭 victim model과 유사할 필요는 없다.
loss function은 crossentropy를 썼다.
